In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3
import jax_dataclasses as jdc
from functools import partial

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *
from sdf_world.network import *

from flax import linen as nn
from flax.training import orbax_utils
import orbax
import pickle
import time

import cyipopt

In [2]:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
restored_grasp = orbax_checkpointer.restore("model/grasp_net_prob_dist")
restored_manip = orbax_checkpointer.restore("model/manip_net_posevec")


to_posevec = lambda x: jnp.hstack([x[4:], SO3(x[:4]).log()])
to_wxyzxyz = lambda x: jnp.hstack([SO3.exp(x[3:]).parameters(), x[:3]])

#grasp net
grasp_net = GraspNet(32)
grasp_fn = lambda x: grasp_net.apply(restored_grasp["params"], x)
def grasp_reconst(g:Array, obj_pose:SE3):
    rot = SO3(grasp_fn(g)[2:]).normalize()
    trans = g/restored_grasp["scale_to_norm"]
    return obj_pose @ SE3.from_rotation_and_translation(rot, trans)
def grasp_reconst_posevec(g, obj_pose):
    rot = SO3(grasp_fn(g)[2:]).normalize()
    trans = g/restored_grasp["scale_to_norm"]
    grasp_pose = obj_pose @ SE3.from_rotation_and_translation(rot, trans)
    return to_posevec(grasp_pose.parameters())
grasp_logit_fn = lambda g: grasp_fn(g)[0]
grasp_dist_fn = lambda g: grasp_fn(g)[1]

In [2]:
world = SDFWorld()
# panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
# panda1 = Robot(world.vis, "panda", panda_model, alpha=0.5)
# panda1.reduce_dim([7, 8], [0.04, 0.04])
# panda2 = Robot(world.vis, "panda", panda_model, alpha=0.5)
# panda2.reduce_dim([7, 8], [0.04, 0.04])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/


In [17]:
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda1 = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda1.reduce_dim([7, 8], [0.04, 0.04])

concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts
concatenating texture: may result in visual artifacts


In [3]:
world.show_in_jupyter()

In [5]:
frame = Frame(world.vis, "frame")

In [6]:
table_lengths = [0.4, 0.4, 0.2]
table_start = Box(world.vis, "table_start", table_lengths, 'white', 0.5)
table_goal = Box(world.vis, "table_goal", table_lengths, 'white', 0.5)
obj_start = Mesh(world.vis, "obj_start", 
                 "./sdf_world/assets/object/mesh.obj",
                 color="blue",
                 alpha=0.5)
obj_goal = Mesh(world.vis, "obj_goal", 
                "./sdf_world/assets/object/mesh.obj",
                color="green",
                 alpha=0.5)

In [7]:
table_start.set_translate([0.5, -0.3, 0.2/2])
table_goal.set_translate([0.5, 0.3, 0.2/2])
obj_lengths = obj_start.mesh.bounding_box.primitive.extents
obj_start.set_translate([0.5, -0.3, obj_lengths[-1]/2+table_lengths[-1]])
trans_goal = jnp.array([0.5, 0.3, obj_lengths[-2]/2+table_lengths[-1]])
obj_goal_pose = SE3.from_rotation_and_translation(
    SO3.from_rpy_radians(jnp.pi/2, 0,0), trans_goal
)
obj_goal.set_pose(obj_goal_pose)

In [8]:
hand_model = RobotModel(HAND_URDF, PANDA_PACKAGE, True)
for link_name, link in hand_model.links.items():
    link.set_surface_points(10)
hand1 = Robot(world.vis, "hand1", hand_model, color="yellow", alpha=0.5)

In [9]:
hand_pc = hand1.get_surface_points_fn(jnp.array([0.04, 0.04]))
hand_pose_wrt_ee = SE3.from_translation(jnp.array([0,0,-0.105]))
env = SDFContainer([table_start, table_goal], 0.05)
# x -> grasps
@jax.jit
def get_hand_points(x, obj_pose1, obj_pose2):
    result = []
    for pose in [obj_pose1, obj_pose2]:
        grasp_pose = grasp_reconst(x, pose)
        hand_base_pose_wrt_world = grasp_pose @ hand_pose_wrt_ee
        assigned_hand_pc = jax.vmap(hand_base_pose_wrt_world.apply)(hand_pc)
        result.append(assigned_hand_pc)
    return jnp.vstack(result)

@jax.jit
def distance_constr(x):
    hand_pcs = get_hand_points(x, obj_start.pose, obj_goal.pose)
    distances = env.distances(hand_pcs)
    top4_indices = jnp.argpartition(distances, 4)[:4]
    return distances[top4_indices]

In [10]:
x = jnp.array([0,0,-0.5])
hand_pcs = get_hand_points(x, obj_start.pose, obj_goal.pose)
pc = PointCloud(world.vis, "hand_pc", hand_pcs, color="red")

In [11]:
# Kinematics
def get_rotvec_angvel_map(v):
    def skew(v):
        v1, v2, v3 = v
        return jnp.array([[0, -v3, v2],
                        [v3, 0., -v1],
                        [-v2, v1, 0.]])
    vmag = jnp.linalg.norm(v)
    vskew = skew(v)
    return jnp.eye(3) \
        - 1/2*skew(v) \
        + vskew@vskew * 1/vmag**2 * (1-vmag/2 * jnp.sin(vmag)/(1-jnp.cos(vmag)))

@jax.jit
def get_ee_fk_jac(q):
    # outputs ee_posevec and analytical jacobian
    fks = panda_model.fk_fn(q)
    p_ee = fks[-1][-3:]
    rotvec_ee = SO3(fks[-1][:4]).log()
    E = get_rotvec_angvel_map(rotvec_ee)
    jac = []
    for posevec in fks[1:8]:
        p_frame = posevec[-3:]
        rot_axis = SE3(posevec).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, p_ee - p_frame)
        jac.append(jnp.hstack([lin_vel, rot_axis]))
    jac = jnp.array(jac).T
    jac = jac.at[3:, :].set(E @ jac[3:, :])
    return jnp.hstack([p_ee, rotvec_ee]), jac

In [30]:
import time
#q = panda1.neutral
# x-> manip
def objective(x):
    frame.set_pose(grasp_reconst(x, obj_start.pose))
    hand_pcs = get_hand_points(x, obj_start.pose, obj_goal.pose)
    pc.reload(points=hand_pcs)
    time.sleep(0.05)
    posevec_grasp = grasp_reconst_posevec(x, obj_start.pose)
    posevec_ee, _ = get_ee_fk_jac(q)
    return 0.5*jnp.sum((posevec_ee[:3] - posevec_grasp[:3])**2)

def gradient(x):
    posevec_grasp = grasp_reconst_posevec(x, obj_start.pose)
    posevec_ee, _ = get_ee_fk_jac(q)
    err = posevec_ee[:3] - posevec_grasp[:3]
    jac = jnp.eye(3)*1/restored_grasp["scale_to_norm"]
    return -jac.T@err

def constraints(x):
    logit = grasp_logit_fn(x)
    dists = distance_constr(x)
    return jnp.hstack([logit, dists])

def jacobian(x):
    jac_logit = jax.grad(grasp_logit_fn)(x)
    jac_dists = jax.jacrev(distance_constr)(x)
    return jnp.vstack([jac_logit, jac_dists])

In [31]:
class Prob:
    pass
prob = Prob()
lb = -np.ones(3)
ub = np.ones(3)
setattr(prob, "objective", objective)
setattr(prob, "gradient", gradient)
setattr(prob, "constraints", constraints)
setattr(prob, "jacobian", jacobian)
ipopt = cyipopt.Problem(
    n=3, m=5,
    problem_obj = prob,
    lb=lb, ub=ub,
    cl=[1., 0.05, 0.05, 0.05, 0.05], 
    cu=[np.inf, np.inf, np.inf, np.inf, np.inf]
)
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option("acceptable_tol", 1.) #release
ipopt.add_option("acceptable_obj_change_tol", 0.01)
ipopt.add_option("acceptable_dual_inf_tol", 1.) 
ipopt.add_option('mu_strategy', 'adaptive')
xsol, info = ipopt.solve(jnp.array([0., -0.5, -0.5]))

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       15
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.2301975e-01 2.83e+01 9.99e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [32]:
import time
g = xsol.copy()

# x-> manip
def objective(x):
    panda1.set_joint_angles(x)
    time.sleep(0.1)

    posevec_grasp = grasp_reconst_posevec(g, obj_start.pose)
    posevec_ee, _ = get_ee_fk_jac(x)
    return 0.5*jnp.sum((posevec_grasp - posevec_ee)**2)

def gradient(x):
    posevec_grasp = grasp_reconst_posevec(g, obj_start.pose)
    posevec_ee, jac_ee = get_ee_fk_jac(x)
    err = posevec_grasp - posevec_ee
    return -jac_ee.T @ err

# def constraints(x):
#     logit = grasp_logit_fn(x)
#     dists = distance_constr(x)
#     return jnp.hstack([logit, dists])

# def jacobian(x):
#     jac_logit = jax.grad(grasp_logit_fn)(x)
#     jac_dists = jax.jacrev(distance_constr)(x)
#     return jnp.vstack([jac_logit, jac_dists])

In [34]:
class Prob:
    pass
prob = Prob()
lb = panda1.lb
ub = panda1.ub
setattr(prob, "objective", objective)
setattr(prob, "gradient", gradient)
# setattr(prob, "constraints", constraints)
# setattr(prob, "jacobian", jacobian)
ipopt = cyipopt.Problem(
    n=7, m=0,
    problem_obj = prob,
    lb=lb, ub=ub,
    #cl=[1., 0.5, 0.05, 0.05, 0.05, 0.05], 
    #cu=[np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
)
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option("acceptable_tol", 0.01) #release
ipopt.add_option("acceptable_obj_change_tol", 0.001)
ipopt.add_option("acceptable_dual_inf_tol", 1.) 
ipopt.add_option('mu_strategy', 'adaptive')
xsol, info = ipopt.solve(q)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.4926333e+00 0.00e+00 2.43e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 